In [27]:
import pandas as pd
import numpy as np
import os

In [28]:
img_types= os.listdir('/kaggle/input/satellite-image-classification/data')
print(img_types)
print('images classes: ', len(img_types))

['cloudy', 'desert', 'green_area', 'water']
images classes:  4


In [29]:
path=('/kaggle/input/satellite-image-classification/data')

In [30]:
images=[]
for item in img_types:
    all_imgs= os.listdir('/kaggle/input/satellite-image-classification/data'+ '/'+ item)

    for img in all_imgs:
        images.append((item, str('/kaggle/input/satellite-image-classification/data' + '/' +item)+ '/' + img))

images[0]

('cloudy',
 '/kaggle/input/satellite-image-classification/data/cloudy/train_17406.jpg')

In [31]:
df= pd.DataFrame(data= images, columns=['label', 'image_path'])
df.head()

,label,image_path
0,cloudy,/kaggle/input/satellite-image-classification/d...
1,cloudy,/kaggle/input/satellite-image-classification/d...
2,cloudy,/kaggle/input/satellite-image-classification/d...
3,cloudy,/kaggle/input/satellite-image-classification/d...
4,cloudy,/kaggle/input/satellite-image-classification/d...


In [32]:
from sklearn.utils import shuffle
df = shuffle(df)
df

,label,image_path
4070,green_area,/kaggle/input/satellite-image-classification/d...
4585,water,/kaggle/input/satellite-image-classification/d...
5199,water,/kaggle/input/satellite-image-classification/d...
1480,cloudy,/kaggle/input/satellite-image-classification/d...
574,cloudy,/kaggle/input/satellite-image-classification/d...
...,...,...
1107,cloudy,/kaggle/input/satellite-image-classification/d...
224,cloudy,/kaggle/input/satellite-image-classification/d...
4814,water,/kaggle/input/satellite-image-classification/d...
1495,cloudy,/kaggle/input/satellite-image-classification/d...


In [33]:
from sklearn.preprocessing import LabelEncoder

#label_encoder = LabelEncoder()
#df['label']= label_encoder.fit_transform(df['label'])
#df['label'].unique()

In [34]:
df

,label,image_path
4070,green_area,/kaggle/input/satellite-image-classification/d...
4585,water,/kaggle/input/satellite-image-classification/d...
5199,water,/kaggle/input/satellite-image-classification/d...
1480,cloudy,/kaggle/input/satellite-image-classification/d...
574,cloudy,/kaggle/input/satellite-image-classification/d...
...,...,...
1107,cloudy,/kaggle/input/satellite-image-classification/d...
224,cloudy,/kaggle/input/satellite-image-classification/d...
4814,water,/kaggle/input/satellite-image-classification/d...
1495,cloudy,/kaggle/input/satellite-image-classification/d...


In [35]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

In [36]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


In [37]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to the size expected by ResNet
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [38]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from sklearn.preprocessing import LabelEncoder

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.data['label'] = self.label_encoder.fit_transform(self.data['label'])
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.data.iloc[idx]['label'], dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

In [39]:
train_dataset = CustomDataset(train_data, transform=transform)
vaild_dataset = CustomDataset(val_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

In [40]:
train_dataset.transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [41]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=vaild_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [42]:
train_loader

In [43]:
train_loader.dataset

In [44]:
for images, labels in train_loader:
      # Your training loop here
      print(f'Batch of images: {images.shape}')
      print(f'Batch of labels: {labels}')
      break  # For demonstration, break after first batch

Batch of images: torch.Size([32, 3, 224, 224])
Batch of labels: tensor([2, 1, 0, 0, 1, 2, 0, 2, 3, 3, 2, 3, 2, 1, 0, 2, 2, 1, 2, 3, 0, 2, 3, 0,
        0, 0, 0, 3, 3, 2, 2, 3])


In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Define a custom LeNet-5-like model based on AlexNet
class LeNet5LikeAlexNet(nn.Module):
    def __init__(self, num_classes=4):
        super(LeNet5LikeAlexNet, self).__init__()
        # Define layers similar to LeNet-5
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),  # Adjusted for 4 classes
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [47]:
# Initialize the model, loss function, and optimizer
num_classes = 4
model = LeNet5LikeAlexNet(num_classes=num_classes)

# Load pre-trained weights from AlexNet
alexnet = models.alexnet(pretrained=True)
state_dict = alexnet.state_dict()
model_dict = model.state_dict()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [49]:
# Freeze layers if needed
for param in model.features.parameters():
    param.requires_grad = False

In [50]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [56]:
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/5], Loss: 0.0814
Epoch [2/5], Loss: 0.1322
Epoch [3/5], Loss: 0.0536
Epoch [4/5], Loss: 0.1025
Epoch [5/5], Loss: 0.0422


In [57]:
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2%}')

In [59]:
evaluate_model(alexnet, test_loader)

Accuracy: 1.15%


In [60]:
torch.save(alexnet.state_dict(), 'alexnet_model.pth')